In [8]:
import cv2
import numpy as np

def ScaleSpace(image,num_scale, num_octave, sigma):
    octaves = []
    gray_image = (cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)).astype(np.float32)
    gray_image_norm = gray_image/255.0
    k = 2 ** (1/(num_scale-1))

    for octave in range(num_octave):
        if octave == 0:
            img= gray_image_norm.copy()
        else:
            img = cv2.pyrDown(octaves[octave-1][0])
    

        scale_space = [img]
        for scale in range(1, num_scale):
            smoothed_image= cv2.GaussianBlur(img, (0,0), sigmaX = sigma*(k**scale),sigmaY = sigma*(k**scale))
            scale_space.append(smoothed_image)
        octaves.append(scale_space)
    
    return octaves






In [9]:
image = r"C:/Users/49179/Desktop/download.png"
image = cv2.imread(image)

In [10]:
octaves = ScaleSpace(image, num_octave=4, num_scale=5, sigma=1.6)

In [12]:
for i in range(4):
    for j in range(5):
        cv2.imshow(f"octave {i+1}, scale {j+1}", octaves[i][j])
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [67]:
import cv2
import numpy as np

def create_scale_space(image, num_octaves=4, num_scales=5, sigma=1.6):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Normalize the image
    gray_image = gray_image.astype(np.float32) / 255.0
    
    octaves = []
    k = 2 ** (1 / (num_scales - 1))
    
    for octave in range(num_octaves):
        # Downsample the image for each octave
        if octave == 0:
            img = gray_image
        else:
            img = cv2.resize(octaves[-1][0], (gray_image.shape[1] // (2 ** octave), gray_image.shape[0] // (2 ** octave)))
        
        scale_space = [img]
        for scale in range(1, num_scales):
            sigma_total = sigma * (k ** scale)
            print(sigma_total)
            img_blurred = cv2.GaussianBlur(img, (0, 0), sigmaX=sigma_total, sigmaY=sigma_total)
            scale_space.append(img_blurred)
        octaves.append(scale_space)
    
    return octaves


def difference_of_gradients(octaves):
    dog_octaves = []
    for octave in octaves:
        dog_octave = []
        for scale in range(1,len(octave)):
            dog = octave[scale] - octave[scale - 1]
            dog_octave.append(dog)
        dog_octaves.append(dog_octave)
    return dog_octaves



In [68]:
octaves = create_scale_space(image)

dog_octaves = difference_of_gradients(octaves)


1.9027313840043538
2.262741699796952
2.6908685288118868
3.1999999999999997
1.9027313840043538
2.262741699796952
2.6908685288118868
3.1999999999999997
1.9027313840043538
2.262741699796952
2.6908685288118868
3.1999999999999997
1.9027313840043538
2.262741699796952
2.6908685288118868
3.1999999999999997


In [23]:
import cv2
import numpy as np

# Assuming 'dog_octaves' is already computed using the Difference of Gaussians

for i in range(len(dog_octaves)):  # Loop through octaves
    for j in range(len(dog_octaves[i])):  # Loop through scales in each octave
        # Prepare the window name
        window_name = f"Octave {i + 1}, Scale {j + 1}"
        # Display the DoG image
        cv2.imshow(window_name, dog_octaves[i][j])
        
        # Optionally, you can adjust the size of the displayed window
        # cv2.resizeWindow(window_name, 300, 300)  # Example size (300x300)
        
        # Wait for a key press before displaying the next image
        cv2.waitKey(0)

        # Close the current window
        cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np
def BaseImage(image, sigma, assumed_blur):
    base_image = cv2.resize(image, (0,0), fx=2,fy=2, interpolation=cv2.INTER_LINEAR)
    total_sigma = np.sqrt(max((sigma**2)-(assumed_blur*2),0.01))
    Upsampled_BaseImage = cv2.GaussianBlur(base_image,(0,0), sigmaX= total_sigma, sigmaY= total_sigma )
    return Upsampled_BaseImage
    
    

In [9]:
image = r"C:/Users/49179/Desktop/download.png"
image = cv2.imread(image)

In [20]:
blurred_image = BaseImage(image, 1.6,0.5)

In [22]:
print(blurred_image.shape)
cv2.imshow("window", blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(3086, 2362, 3)


In [36]:
int(round(np.log(min(blurred_image.shape[:2])) / np.log(2) - 1))

10

In [38]:
int(round(np.log(2362)/np.log(2)-1))

10

In [39]:
int(round(np.log2(2362/10)))+ 1

9

In [43]:
int(np.log2(2362 / np.log(2)-1)) + 1

12

In [61]:
sigma = np.sqrt(2)/2
k =  2**(1. /2)
kernels = np.zeros(5)
kernels = [np.sqrt(2)]
for scale in range(1,5):
        sigma_prev = sigma * (k**(scale - 1))
        sigma_total = k * sigma_prev
        kernels.append(np.sqrt(sigma_total**2 - sigma_prev**2))
            

In [62]:
kernels

[np.float64(1.4142135623730951),
 np.float64(0.7071067811865478),
 np.float64(1.0000000000000002),
 np.float64(1.4142135623730954),
 np.float64(2.0000000000000004)]

In [70]:
num_images_per_octave = 5
k = 2 ** (1. / 2)
gaussian_kernels = np.zeros(num_images_per_octave)  # scale of gaussian blur necessary to go from one blur scale to the next within an octave
gaussian_kernels[0] = sigma

for image_index in range(1, num_images_per_octave):
        sigma_previous = (k ** (image_index - 1)) * sigma
        sigma_total = k * sigma_previous
        gaussian_kernels[image_index] = np.sqrt(sigma_total ** 2 - sigma_previous ** 2)
gaussian_kernels

array([0.70710678, 0.70710678, 1.        , 1.41421356, 2.        ])

In [58]:
0.707**2

0.49984899999999993

In [66]:
import numpy as np

class GaussianKernelGenerator:
    def __init__(self, s, sigma):
        self.s = s  # Number of intervals per octave
        self.sigma = sigma  # Base sigma for the base image

    def CreateKernels(self):
        k = 2**(1. / self.s)
        num_image_per_octave = self.s + 3
        kernels = np.zeros(num_image_per_octave)
        kernels[0] = self.sigma

        for scale in range(1, num_image_per_octave+1):
            sigma_prev = (k**(scale - 1)) * self.sigma
            sigma_total = k * sigma_prev
            kernels[scale] = np.sqrt(sigma_total**2 - sigma_prev**2)
        
        return kernels

# Example usage:
generator = GaussianKernelGenerator(s=2, sigma=0.707)
kernels = generator.CreateKernels()
print(kernels)


IndexError: index 5 is out of bounds for axis 0 with size 5

In [5]:
def generateBaseImage(image, sigma, assumed_blur):
    image = cv2.resize(image, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    sigma_diff = np.sqrt(max((sigma ** 2) - ((2 * assumed_blur) ** 2), 0.01))
    return cv2.GaussianBlur(image, (0, 0), sigmaX=sigma_diff, sigmaY=sigma_diff)  # the image blur is now sigma instead of assumed_blur

def computeNumberOfOctaves(image_shape):
    """Compute number of octaves in image pyramid as function of base image shape (OpenCV default)
    """
    return int(round(np.log(min(image_shape)) / np.log(2) - 1))

def generateGaussianKernels(sigma, num_intervals):
    """Generate list of gaussian kernels at which to blur the input image. Default values of sigma, intervals, and octaves follow section 3 of Lowe's paper.
    """
    num_images_per_octave = num_intervals + 3
    k = 2 ** (1. / num_intervals)
    gaussian_kernels = np.zeros(num_images_per_octave)  # scale of gaussian blur necessary to go from one blur scale to the next within an octave
    gaussian_kernels[0] = sigma

    for image_index in range(1, num_images_per_octave):
        sigma_previous = (k ** (image_index - 1)) * sigma
        sigma_total = k * sigma_previous
        gaussian_kernels[image_index] = np.sqrt(sigma_total ** 2 - sigma_previous ** 2)
    return gaussian_kernels

def generateGaussianImages(image, num_octaves, gaussian_kernels):
    """Generate scale-space pyramid of Gaussian images
    """

    gaussian_images = []

    for octave_index in range(num_octaves):
        gaussian_images_in_octave = []
        gaussian_images_in_octave.append(image)  # first image in octave already has the correct blur
        for gaussian_kernel in gaussian_kernels[1:]:
            image = cv2.GaussianBlur(image, (0, 0), sigmaX=gaussian_kernel, sigmaY=gaussian_kernel)
            gaussian_images_in_octave.append(image)
        gaussian_images.append(gaussian_images_in_octave)
        octave_base = gaussian_images_in_octave[-3]
        image = cv2.resize(octave_base, (int(octave_base.shape[1] / 2), int(octave_base.shape[0] / 2)), interpolation=cv2.INTER_NEAREST)
    return np.array(gaussian_images)

In [8]:
image = r"C:/Users/49179/Desktop/download.png"
image = cv2.imread(image)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Normalize the image
gray_image = gray_image.astype(np.float32) / 255.0
def GI(image, sigma=1.6, num_intervals=3, assumed_blur=0.5, image_border_width=5):

    image = image.astype('float32')
    base_image = generateBaseImage(image, sigma, assumed_blur)
    num_octaves = computeNumberOfOctaves(base_image.shape)
    gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
    gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
    return gaussian_images

In [9]:
h = GI(gray_image)
for octave, images in enumerate(h):
        for i, image in enumerate(images):
            cv2.imshow(f'Octave {octave+1}, Gaussian Kernel {i+1}', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (10, 6) + inhomogeneous part.

In [24]:
import numpy as np
import cv2
import logging
from math import sqrt, log

# Configure logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

def generateBaseImage(image, sigma, assumed_blur):
    """Generate base image from input image by upsampling by 2 in both directions and blurring."""
    logger.debug('Generating base image...')
    image = cv2.resize(image, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    sigma_diff = sqrt(max((sigma ** 2) - ((2 * assumed_blur) ** 2), 0.01))
    return cv2.GaussianBlur(image, (0, 0), sigmaX=sigma_diff, sigmaY=sigma_diff)

def computeNumberOfOctaves(image_shape):
    """Compute number of octaves in image pyramid as function of base image shape (OpenCV default)."""
    return int(round(log(min(image_shape)) / log(2) - 1))

def generateGaussianKernels(sigma, num_intervals):
    num_images_per_octave = num_intervals + 3
    k = 2 ** (1. / num_intervals)
    gaussian_kernels = np.zeros(num_images_per_octave)
    gaussian_kernels[0] = sigma

    for image_index in range(1, num_images_per_octave):
        sigma_previous = (k ** (image_index - 1)) * sigma
        sigma_total = k * sigma_previous
        gaussian_kernels[image_index] = sqrt(sigma_total ** 2 - sigma_previous ** 2)
    return gaussian_kernels

def generateGaussianImages(image, num_octaves, gaussian_kernels):
    gaussian_images = []

    for octave_index in range(num_octaves):
        gaussian_images_in_octave = [image]  # first image in octave already has the correct blur
        for gaussian_kernel in gaussian_kernels[1:]:
            image = cv2.GaussianBlur(gaussian_images_in_octave[-1], (0, 0), sigmaX=gaussian_kernel, sigmaY=gaussian_kernel)
            gaussian_images_in_octave.append(image)
        gaussian_images.append(gaussian_images_in_octave)
        octave_base = gaussian_images_in_octave[-3]
        image = cv2.resize(octave_base, (int(octave_base.shape[1] / 2), int(octave_base.shape[0] / 2)), interpolation=cv2.INTER_NEAREST)
    return gaussian_images

if __name__ == '__main__':
    image_path = r"C:/Users/49179/Desktop/12.jpg"
    image = cv2.imread(image_path)
    if image is None:
        print("error")
    else:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
        base_image = generateBaseImage(gray_image, sigma=1.6, assumed_blur=0.3)

        # Define the number of octaves and the Gaussian kernels
        num_octaves = computeNumberOfOctaves(base_image.shape)
        kernels = generateGaussianKernels(sigma=1.6, num_intervals=3)
        print(kernels)

        # Generate Gaussian images
        gaussian_images = generateGaussianImages(base_image, num_octaves, kernels)
        

        # Display the Gaussian pyramid
        '''for octave_index, images_in_octave in enumerate(gaussian_images):
            for image_index, img in enumerate(images_in_octave):
                cv2.imshow(f'Octave {octave_index + 1}, Image {image_index + 1}', img / 255.0)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
'''

[1.6        1.2262735  1.54500779 1.94658784 2.452547   3.09001559]


In [27]:
def DoG(gaussian_images):
        DoG_pyramid = []
        for octave, images in enumerate(gaussian_images):
            DoG_octave = []
            for scale in range(1,len(images)):
                difference = images[scale] - images[scale-1]
                DoG_octave.append(difference)
            DoG_pyramid.append(DoG_octave)
        return DoG_pyramid
DoG(gaussian_images)

[[array([[1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00],
         [1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00],
         [1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00],
         ...,
         [1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00],
         [1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00],
         [1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00]], dtype=float32),
  array([[1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00],
         [1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00],
         [1.5258789e-05, 1.5258789e-05, 1

In [23]:
import cv2
import numpy as np
from math import sqrt, log

def CreateBaseImage(image, sigma, assumed_blur):
    image = cv2.resize(image, (0,0), fx=2,fy=2, interpolation=cv2.INTER_LINEAR)
    sigma_total = sqrt(max((sigma**2) - ((2*assumed_blur)**2),0.01))
    base_image = cv2.GaussianBlur(image, (0,0), sigmaX=sigma_total, sigmaY=sigma_total)
    return base_image

def Number_of_octaves(image):
    return int(round(log(min(image.shape[:2]))/log(2))-1)
'''
def CreateGaussianKernels(num_scale, sigma):
    num_images_per_octave = num_scale + 3
    k = 2 ** (1. / num_scale)
    Gaussian_kernels = np.zeros(num_images_per_octave)
    Gaussian_kernels[0] = sigma

    for scale in range(1, num_images_per_octave):
        sigma_prev = (k ** (scale -1)) * sigma
        sigma_current = k* sigma_prev
        Gaussian_kernels[scale] = sqrt(sigma_current**2 - sigma_prev**2)
    return Gaussian_kernels'''

def CreateGaussianKernels(sigma, num_intervals):
    num_images_per_octave = num_intervals + 3
    print(num_images_per_octave)
    k = 2 ** (1. / num_intervals)
    gaussian_kernels = np.zeros(num_images_per_octave)
    gaussian_kernels[0] = sigma

    for image_index in range(1, num_images_per_octave):
        sigma_previous = (k ** (image_index - 1)) * sigma
        sigma_total = k * sigma_previous
        gaussian_kernels[image_index] = sqrt(sigma_total ** 2 - sigma_previous ** 2)
    return gaussian_kernels


image_path = r"C:/Users/49179/Desktop/12.jpg"
image = cv2.imread(image_path)
if image is None:
    logger.error(f"Image not found at {image_path}")
else:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
    base_image = CreateBaseImage(gray_image, sigma=1.6, assumed_blur=0.3)
    print(Number_of_octaves(base_image))
    print(CreateGaussianKernels(1.6,3))

cv2.imshow("window", base_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

8
6
[1.6        1.2262735  1.54500779 1.94658784 2.452547   3.09001559]


In [8]:
def generateBaseImage(image, sigma, assumed_blur):
    """Generate base image from input image by upsampling by 2 in both directions and blurring."""

    image = cv2.resize(image, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    sigma_diff = np.sqrt(max((sigma ** 2) - ((2 * assumed_blur) ** 2), 0.01))
    return cv2.GaussianBlur(image, (0, 0), sigmaX=sigma_diff, sigmaY=sigma_diff)
image_path = r"C:/Users/49179/Desktop/12.jpg"
image = cv2.imread(image_path)
if image is None:
    logger.error(f"Image not found at {image_path}")
else:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
    base_image = generateBaseImage(gray_image, sigma=1.6, assumed_blur=0.3)

cv2.imshow("window", base_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
import cv2
from math import sqrt,log
import numpy as np
import logging as logger
def CreateBaseImage(image, sigma, assumed_blur):
    logger.debug('Creating base image...')
    image = cv2.resize(image, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    sigma_total = sqrt(max((sigma ** 2) - ((2 * assumed_blur) ** 2), 0.01))
    base_image = cv2.GaussianBlur(image, (0, 0), sigmaX=sigma_total, sigmaY=sigma_total)
    return base_image

def Number_of_octaves(image):
    return int(round(log(min(image.shape[:2])) / log(2) - 1))

# Example usage
if __name__ == "__main__":
    image_path = "C:/Users/49179/Desktop/12.jpg"
    image = cv2.imread(image_path)
    if image is None:
        logger.error(f"Image not found at {image_path}")
    else:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
        base_image = CreateBaseImage(gray_image, sigma=1.6, assumed_blur=0.3)
        num_octaves = Number_of_octaves(base_image)
        print(num_octaves)
cv2.imshow("window", base_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

8
